In [6]:
from skimage import feature
from skimage.color import rgb2gray
from skimage.transform import resize
from skimage.morphology import erosion, dilation, rectangle, square
from skimage.measure import find_contours

from scipy import ndimage as ndi
from scipy.misc import imsave

%matplotlib inline

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

from collections import defaultdict
from copy import deepcopy

#import pyscreenshot as pysc
from PIL import ImageGrab
# from PIL import Image

import random
import math
from time import sleep

In [7]:
colormode = 'L'
mystery = ndi.imread('grid_square_mystery.png', mode=colormode)
empty = ndi.imread('grid_square_empty.png',  mode=colormode)
grid_one = ndi.imread('grid_one.png', mode=colormode)
grid_two = ndi.imread('grid_two.png', mode=colormode)
grid_three = ndi.imread('grid_three.png', mode=colormode)
grid_four = ndi.imread('grid_four.png', mode=colormode)
grid_five = ndi.imread('grid_five.png', mode=colormode)
#minesweep = ndi.imread('My_shot.png', mode='L')
flag = ndi.imread('flag.png', mode=colormode)
smiley_face = ndi.imread('smiley_face.png', mode=colormode)
frowny_face = ndi.imread('frowny_face.png', mode=colormode)
glasses_face = ndi.imread('glasses_face.png', mode=colormode)
difficulty_level = 1

In [8]:
if difficulty_level == 1:
    x_left = -275
    x_right = 335
    y_top = -85
    y_bot = 690
    grid_rows = 9
    grid_cols = 9
elif difficulty_level == 2:
    x_left = -490
    x_right = 545
    y_top = -85
    y_bot = 1100
    grid_rows = 16
    grid_cols = 16

In [9]:
def dist((x1,y1),(x2,y2)):
    return math.sqrt((x2-x1)**2 + (y2-y1)**2)

In [10]:
def remove_close_points(spots, similarity_matrix, threshold = 5):
    spotscopy = deepcopy(spots)
    for i in range(len(spotscopy)):
        for j in range(len(spotscopy)):
            a = spotscopy[i]
            b = spotscopy[j]
            if dist(a,b) < threshold and j > i:
                if similarity_matrix[a[0]][a[1]] > similarity_matrix[b[0]][b[1]] and b in spots:
                    spots.remove(b)
                elif similarity_matrix[a[0]][a[1]] <= similarity_matrix[b[0]][b[1]] and a in spots:
                    spots.remove(a)
    return spots

In [11]:
def find_candidate_locations(similarity_matrix):
    candidate_spots = []
    thing = np.where(similarity_matrix > .95)
#     for n in range(len(similarity_matrix)):
#         for m in range(len(similarity_matrix[n])):
#             if similarity_matrix[n][m] > .95:
#                 candidate_spots.append((n,m))
    return zip(thing[0], thing[1])
#     final_spots = remove_close_points(candidate_spots, similarity_matrix)
#     return final_spots

In [12]:
from Quartz.CoreGraphics import CGEventCreateMouseEvent
from Quartz.CoreGraphics import CGEventPost
from Quartz.CoreGraphics import kCGEventMouseMoved
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseDown
from Quartz.CoreGraphics import kCGEventLeftMouseUp
from Quartz.CoreGraphics import kCGMouseButtonLeft
from Quartz.CoreGraphics import kCGHIDEventTap
from Quartz.CoreGraphics import kCGEventRightMouseDown
from Quartz.CoreGraphics import kCGEventRightMouseUp

def mouseEvent(type, posx, posy):
        theEvent = CGEventCreateMouseEvent(
                    None, 
                    type, 
                    (posx,posy), 
                    kCGMouseButtonLeft)
        CGEventPost(kCGHIDEventTap, theEvent)

def mousemove(posx,posy):
        mouseEvent(kCGEventMouseMoved, posx,posy);

def mouseclick(posx,posy):
        # uncomment this line if you want to force the mouse 
        # to MOVE to the click location first (I found it was not necessary).
        #mouseEvent(kCGEventMouseMoved, posx,posy);
        mouseEvent(kCGEventLeftMouseDown, posx,posy);
        mouseEvent(kCGEventLeftMouseUp, posx,posy);

def rightmouseclick(posx,posy):
        mouseEvent(kCGEventRightMouseDown, posx,posy);
        mouseEvent(kCGEventRightMouseUp, posx,posy);

In [13]:
#mouseclick(max_loc[0]/2 + 100, max_loc[1]/2 + 50)

In [14]:
#rightmouseclick(980,500)

In [15]:
def generate_grid(picture):
    my_grid = np.zeros((grid_rows,grid_cols))
    #gridss = cv2.matchTemplate(fs, picture, cv2.TM_CCOEFF_NORMED)
    ones = cv2.matchTemplate(picture, grid_one, cv2.TM_CCOEFF_NORMED)
    twos = cv2.matchTemplate(picture, grid_two, cv2.TM_CCOEFF_NORMED)
    threes = cv2.matchTemplate(picture, grid_three, cv2.TM_CCOEFF_NORMED)
    fours = cv2.matchTemplate(picture, grid_four, cv2.TM_CCOEFF_NORMED)
    fives = cv2.matchTemplate(picture, grid_five, cv2.TM_CCOEFF_NORMED)
    empties = cv2.matchTemplate(picture, empty, cv2.TM_CCOEFF_NORMED)
    mysteries = cv2.matchTemplate(picture, mystery, cv2.TM_CCOEFF_NORMED)
    flags = cv2.matchTemplate(picture, flag, cv2.TM_CCOEFF_NORMED)
    smileys = cv2.matchTemplate(picture, smiley_face, cv2.TM_CCOEFF_NORMED)
    frownys = cv2.matchTemplate(picture, frowny_face, cv2.TM_CCOEFF_NORMED)
    glassys = cv2.matchTemplate(picture, glasses_face, cv2.TM_CCOEFF_NORMED)

    adj_x = 4
    adj_y = 4
    smiley_max = cv2.minMaxLoc(smileys)[1]
    frowny_max = cv2.minMaxLoc(frownys)[1]
    glasses_max = cv2.minMaxLoc(glassys)[1]
    winorlose = np.argmax([smiley_max,frowny_max,glasses_max])
    if winorlose > 0:
        return []
    #if gridss[max_loc[1] + y_top, max_loc[0] + x_left] < .5:
        #return []
        

    #there is a clear path to rid ourselves of this for loop. Implement later.
    #I am looking back and have no idea how to rid myself of this for loop.
    for loc in grid_locations:
        coords = grid_loc_dict[loc]
        one_sim = ones[loc[0] + adj_x, loc[1] + adj_y]
        two_sim = twos[loc[0] + adj_x, loc[1] + adj_y]
        three_sim = threes[loc[0] + adj_x, loc[1] + adj_y]
        four_sim = fours[loc[0] + adj_x, loc[1] + adj_y]
        five_sim = fives[loc[0] + adj_x, loc[1] + adj_y]
        empty_sim = empties[loc[0] + adj_x, loc[1] + adj_y]
        mystery_sim = mysteries[loc[0], loc[1]]
        flag_sim = flags[loc[0], loc[1]]
        ind = np.argmax([empty_sim, one_sim, two_sim, three_sim, four_sim, five_sim, mystery_sim, flag_sim])
        if ind == 7:
            idx = -3
        elif ind == 6:
            idx = -1
        else:
            idx = ind
        #print my_grid
        #print coords
        my_grid[coords[0], coords[1]] = idx
    return my_grid

In [16]:
def finalize_grid_locations():
    fullscreen = ImageGrab.grab().convert(colormode)# X1,Y1,X2,Y2
    global fs
    fs = np.array(fullscreen)
    simil_mat = cv2.matchTemplate(fs, smiley_face, cv2.TM_CCOEFF_NORMED)
    thing = cv2.minMaxLoc(simil_mat)
    max_loc = thing[3]
    mines = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
    #mines.show()
    mines = np.array(mines)
    mysteries = cv2.matchTemplate(mines, mystery, cv2.TM_CCOEFF_NORMED)
    grid_locations = remove_close_points(find_candidate_locations(mysteries), mysteries)
    return max_loc, grid_locations

In [12]:
def click_on_square(coords):
    click_target_x = (max_loc[0] + x_left + grid_coord_dict[coords][1])/2 + 15
    click_target_y= (max_loc[1] + y_top + grid_coord_dict[coords][0])/2 + 15
    mouseclick(max_loc[0]/2 -10, max_loc[1]/2)
    mouseclick(click_target_x, click_target_y)

In [13]:
def right_click_on_square(coords):
    if grid[coords[0],coords[1]] != -1:
        print grid[coords[0],coords[1]]
    click_target_x = (max_loc[0] + x_left + grid_coord_dict[coords][1])/2 + 15
    click_target_y= (max_loc[1] + y_top + grid_coord_dict[coords][0])/2 + 15
    mouseclick(max_loc[0]/2 -10, max_loc[1]/2)
    rightmouseclick(click_target_x,click_target_y)
    #rightmouseclick(click_target_x,click_target_y)

In [14]:
def one_square_think(coords):
    did_something = False
    flag_count = count_adjacent(coords, -3)
    mystery_count = count_adjacent(coords,-1)
    if grid[coords] == flag_count and mystery_count > 0:
        click_all_adjacent_mysteries(coords)
        did_something = True
    elif grid[coords] - flag_count == mystery_count and mystery_count > 0:
        right_click_all_adjacent_mysteries(coords)
        did_something = True
    return did_something
    
    

In [15]:
def click_all_adjacent_mysteries(coords):
    p_clicks = np.where(grid == -1)
    p_clicks = zip(p_clicks[0], p_clicks[1])
    for x,y in p_clicks:
        if x >= coords[0] - 1 and x <= coords[0] + 1 and y >= coords[1] -1 and y <= coords[1] + 1 and coords != (x,y): 
            click_on_square((x,y))

In [16]:
def right_click_all_adjacent_mysteries(coords):
    p_clicks = np.where(grid == -1)
    p_clicks = zip(p_clicks[0], p_clicks[1])
    for x,y in p_clicks:
        if x >= coords[0] - 1 and x <= coords[0] + 1 and y >= coords[1] -1 and y <= coords[1] + 1 and coords != (x,y): 
            right_click_on_square((x,y))
            grid[(x,y)] = -3

In [17]:
def count_adjacent(coords, num):
    i = coords[0] + 1
    j = coords[1] + 1
    temp_grid = np.multiply(np.ones((grid.shape[0] + 2, grid.shape[1] + 2)),-10)
    temp_grid[1:grid.shape[0] + 1, 1:grid.shape[1] + 1] = deepcopy(grid)
    adj = np.concatenate((temp_grid[i+1][j-1:j+2],temp_grid[i][j-1:j+2],temp_grid[i-1][j-1:j+2]))
    #print adj
    adj = np.delete(adj,4)
    return sum(adj == num)

In [18]:
def naive_algorithm():
    did_something = False
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            coords = (i,j)
            if one_square_think(coords) == True:
                did_something = True
                print 'strategic clicking'
            
    if not did_something:
        print 'im clicking randomly!'
        #print grid
        mystery_squares = np.where(grid == -1)
        #print mystery_squares
        mystery_squares = zip(mystery_squares[0], mystery_squares[1])
        n = random.randint(0, len(mystery_squares) - 1)
        click_on_square(mystery_squares[n])     
    return True
    

In [19]:
import subprocess



In [76]:
def play_minesweeper():
#     subprocess.call(
#     ["/usr/bin/open", "-n", "-a", "/Applications/Minesweeper Deluxe.app"]
#     )
#     sleep(5)
    global max_loc
    #print max_loc
    global grid_locations
    max_loc, grid_locations = finalize_grid_locations()
    #print grid_locations
    #create dictionary between locations and coordinates
    
    global grid_loc_dict
    grid_loc_dict = dict()
    for i in range(len(grid_locations)):
        tup = grid_locations[i]
        grid_loc_dict[tup] = (i/grid_rows, i % grid_rows)

    global grid_coord_dict
    grid_coord_dict = dict()
    for x in grid_loc_dict.items():
        grid_coord_dict[x[1]] = x[0]
    global grid
    grid = np.negative(np.ones((grid_rows,grid_cols)))
    
    while(len(grid) > 0):
        ble = naive_algorithm()
        #sleep(.05)
        mines2 = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
        #mines2.show()
        mines2 = np.array(mines2)
        grid = generate_grid(mines2)
        print grid

In [17]:
max_loc, grid_locations = finalize_grid_locations()
    #print grid_locations
    #create dictionary between locations and coordinates
    
global grid_loc_dict
grid_loc_dict = dict()
for i in range(len(grid_locations)):
        tup = grid_locations[i]
        grid_loc_dict[tup] = (i/grid_rows, i % grid_rows)
    
global grid_coord_dict
grid_coord_dict = dict()
for x in grid_loc_dict.items():
    grid_coord_dict[x[1]] = x[0]
global grid
grid = np.negative(np.ones((grid_rows,grid_cols)))

In [19]:
mines2 = ImageGrab.grab(bbox = (max_loc[0] + x_left, max_loc[1] + y_top, max_loc[0] + x_right, max_loc[1] + y_bot)).convert(colormode)  # X1,Y1,X2,Y2
mines2.show()
mines2 = np.array(mines2)
grid = generate_grid(mines2)

In [20]:
grid

array([[ 0.,  0.,  0.,  1., -1.,  1.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  1.,  1.,  1.,  0.,  1., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  2., -1.],
       [ 1.,  1.,  1.,  1.,  1.,  0.,  1., -1., -1.],
       [-1., -1., -1., -1.,  1.,  0.,  1.,  1., -1.],
       [-1., -1., -1., -1.,  1.,  0.,  0.,  1., -1.],
       [-1., -1., -1., -1.,  2.,  2.,  1.,  1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.],
       [-1., -1., -1., -1., -1., -1., -1., -1., -1.]])

In [77]:
play_minesweeper()

im clicking randomly!
[[-1. -1.  1.  0.  1. -1. -1. -1. -1.]
 [-1. -1.  1.  0.  2. -1. -1. -1. -1.]
 [-1. -1.  2.  0.  2. -1. -1. -1. -1.]
 [-1. -1.  1.  0.  1.  2.  2.  1. -1.]
 [ 1.  1.  1.  0.  0.  0.  0.  1. -1.]
 [ 0.  0.  0.  0.  1.  1.  1.  1. -1.]
 [ 0.  0.  0.  0.  1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  1.  1.  2. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  1. -1. -1.]]
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
[[-1. -1.  1.  0.  1. -1. -1. -1. -1.]
 [-1. -1.  1.  0.  2. -3. -1. -1. -1.]
 [-1. -1.  2.  0.  2. -3. -3.  2.  1.]
 [-1. -3.  1.  0.  1.  2.  2.  1.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  1.  1.  1.  1. -1.]
 [ 0.  0.  0.  0.  1. -3.  2.  2. -1.]
 [ 0.  0.  0.  0.  1.  1.  2. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  1. -1. -1.]]
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategic clicking
strategi

In [ ]:
i = 1
j = 1
temp_grid = np.multiply(np.ones((grid.shape[0] + 2, grid.shape[1] + 2)),-2)
temp_grid[1:grid.shape[0] + 1, 1:grid.shape[1] + 1] = grid

In [ ]:
adj = np.concatenate((temp_grid[i+1][j-1:j+2],temp_grid[i][j-1:j+2],temp_grid[i-1][j-1:j+2]))
adj = np.delete(adj,4)

In [ ]:
adj

In [ ]:
temp_grid

In [ ]:
count_adjacent(a, (1,1), 8)


In [ ]:
np.multiply(temp_grid, -2)

In [70]:
a =np.array([[3,4,5],[1,2,3],[7,8,9]])

In [74]:
a

array([[3, 4, 5],
       [1, 2, 3],
       [7, 8, 9]])

In [75]:
a[(1,1),(0,0)]

array([1, 1])

In [ ]:
np.reshape(a,(1,9))

In [ ]:
b = np.

In [ ]:
b = np.ones((a.shape[0] + 2, a.shape[1] + 2))

b[1:a.shape[0] + 1, 1:a.shape[1] + 1] = a

In [ ]:
b

In [ ]:
len(np.where(a == 3))


In [ ]:
cv2.minMaxLoc(np.array([[1,0],[0,1]]))

In [ ]:
thing = np.concatenate((a[0][0:3], a[1][0:3], a[2][0:3]))

In [ ]:
np.delete(thing,4)

In [ ]:
np.reshape(thing, (3,3))


In [ ]:
a == 4

In [ ]:
a[(0,1)]

In [ ]:
def testy(c,d):
    global r
    r = c
    printytesty()
    

In [ ]:
def printytesty():
    print r

In [ ]:
testy(1,2)

In [ ]:
a

In [69]:
a

NameError: name 'a' is not defined